In [1]:
import pennylane as qml
from pennylane import numpy as np

In [14]:
dev = qml.device("default.qubit", wires = 3)

In [15]:
np.random.seed(1)

Create a quantum Circuit.

In [21]:
@qml.qnode(dev)
def circuit(params):
    qml.RX(params[0], wires = 0)
    qml.RY(params[1], wires = 1)
    qml.RZ(params[2], wires = 2)
    
    qml.broadcast(qml.CNOT, wires = [0,1,2], pattern = "ring")
    #qml.broadcast applies unitary gates multiple times according "pattern"
    
    return qml.expval(qml.PauliY(0) @ qml.PauliZ(2))

In [23]:
params = np.random.rand(3)
my_circuit = circuit(params)
print("measurement value of the circuit: " + str(circuit(params)))
print(circuit.draw())

measurement value of the circuit: -0.2518328826855954
 0: ──RX(0.67)───╭C──────╭X──╭┤ ⟨Y ⊗ Z⟩ 
 1: ──RY(0.417)──╰X──╭C──│───│┤         
 2: ──RZ(0.559)──────╰X──╰C──╰┤ ⟨Y ⊗ Z⟩ 



Find the gradient of the circuit.

In [27]:
grad_function = qml.grad(circuit)
print(grad_function(params))

(array([-0.31756367, -0.56803077,  0.        ]),)


Implement Parameter Shift Rule.

In [24]:
def parameter_shift_term(qnode, params, i):
    shifted = params.copy()
    shifted[i] += np.pi / 2
    forward = qnode(shifted)
    
    shifted = params.copy()
    shifted[i] -= np.pi / 2
    backward = qnode(shifted)
    
    return 0.5 * (forward - backward)

In [25]:
def parameter_shift(qnode, params):
    gradients = np.zeros_like(params)    
    for i in range(len(params)):
        gradients[i] = parameter_shift_term(qnode, params, i)
        
    return gradients

In [26]:
print(parameter_shift(circuit, params))

[-0.31756367 -0.56803077  0.        ]


For parameter shift rule it requires 2 * n quantum circuit evaluation for n parameters.